<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" />
</div>

# Greenberger–Horne–Zeilinger State-Fidelity Calculation on System Model H2 with 56 qubits

Quantinuum has released an upgrade to the second generation H-Series device, System Model H2. This upgrade increases the number of qubits on the device from 32 qubits to 56 qubits. The technical specifications for the System Model H2 devices are available [here](https://www.quantinuum.com/hardware/h2). At 56 qubits, full statevector simulation is classically intractable. This signals the need to develop more advanced compute tools for simulations involving more than 40 qubits. Stabilizer simulation can be used (1000-qubit limit), but this constrains the jobs to only utilize the Clifford gate set.

This knowledge articles details the generation of the Greenberger–Horne–Zeilinger (GHZ) on the 56-qubit `H2-1` device and the use of available emulation resources to optimize job preparation and submission. Producing GHZ states is a demanding test of qubit coherence, as they are maximally sensitive probes of global dephasing. Measuring the GHZ state fidelity is a useful test for measuring the quality and performance of the `H2-1` device. The logarithmic-depth GHZ state preparation protocol [[arxiv.2305.03828](https://arxiv.org/abs/2305.03828)] is used. The syntax checker is used to to verify the code is executable on the `H2-1` device.

**Contents**

* [GHZ State Fidelity](GHZ-State-Fidelity)
* [Emulation Resources for 56 Qubit Jobs](Emulation-Resources-for-56-Qubit-Jobs)
* [Analysis of GHZ State Fidelity after Execution](Analysis-of-GHZ-State-Fidelity-after-Execution)

## GHZ State Fidelity

The $N$-qubit GHZ state preparation, using the definition,
\begin{equation}
| {GHZ}_N \rangle = \frac{1}{2} \left( {| 0 \rangle}^{\bigotimes N} + {| 1 \rangle}^{\bigotimes N} \right),
\end{equation}
where $N=56$. A logarithmic-depth synthesis method is used [[arxiv.1807.05572](https://arxiv.org/abs/1807.05572)], an improvement over the standard GHZ circuit construction.

In [ ]:
import numpy as np
from pytket.circuit import Circuit

def logarithmic_ghz_circuit(n_qubits):
    n = n_qubits
    circ = Circuit(n,n)
    # construct GHZ prep circuit, optimized for parallel gates
    circ.H(0)
    for i in range(int(np.ceil(np.log2(n)))):
        for j in range(2**i):
            if j+2**i < n:
                circ.CX(j,j+2**i)
    return circ

In [ ]:
ghz_circuit = logarithmic_ghz_circuit(56)

The circuit is visualised using the `pytket.circuit.display` module.

In [ ]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(ghz_circuit)

To estimate the GHZ state fidelity, $F$, the method from [arxiv.0706.2432](https://arxiv.org/pdf/0706.2432) is used,
\begin{equation}
F =  P + \chi,
\end{equation}
where $P$ is the GHZ population and $\chi$ is the GHZ parity. These are quantities that need to be measured on the quantum hardware. The GHZ population, $P$, requires measurements in the computational basis on every qubit, after the GHZ state has been prepared, and is defined as,
\begin{equation}
P = \frac{1}{2} \textrm{Tr} \left( \rho | 0 \rangle^{\bigotimes N} \langle 0 |^{\bigotimes N} \right) + \frac{1}{2} \textrm{Tr} \left( \rho | 1 \rangle^{\bigotimes N} \langle 1 |^{\bigotimes N} \right)
\end{equation},
where $\rho$ is the density matrix with respect to the GHZ state.

In [ ]:
from pytket.circuit import Circuit

def generate_population_circuit(
    ghz_circuit: Circuit, 
) -> Circuit:
    circuit = ghz_circuit.copy()
    circuit.add_barrier(circuit.qubits)
    circuit.measure_all()
    return circuit

In [ ]:
population_circuit = generate_population_circuit(ghz_circuit)

Once the measurements are collected from the `H2-1` device, the outcomes are filtered into three buckets:
* all-zeros readouts (a bitstring containing `0`s) 
* all-ones readouts (a bitstring containing `1`s) 
* undesired mixed readouts

In an ideal setting, both all-ones and all-zeros bitstrings are equally likely. The population quantifies the amount of error arising from inherent device noise.

In [ ]:
from typing import Dict
from pytket.backends.backendresult import BackendResult

def compute_ghz_populations(
    backend_result: BackendResult
) -> Dict[str, float]:
    distribution = backend_result.get_distribution()
    n = len(backend_result.c_bits)
    zero_string = (0, )*n
    ones_string = (1, )*n
    zeros = distribution.get(zero_string, 0)
    ones = distribution.get(ones_string, 0)
    alt = sum([probability for bitstring, probability in distribution.items() if bitstring != zero_string and bitstring != ones_string]) 
    return {"zeros": zeros, "ones": ones, "something else": alt}

The GHZ parity, $\chi$, requires each qubit to measured in the $X$-$Y$ plane of the Bloch sphere. The parity, $\chi$, requires $N$ circuits to be measured. For a 56-qubit problem, 56 circuits are generated and require non-clifford gates. $\chi$ is defined as,
\begin{equation}
\chi \left( k \right)= \frac{1}{N} \sum_{k=1}^{N} (-1)^{k} \left[ \cos \left( \frac{k \pi}{N} \right) + \sin \left( \frac{k \pi}{N} \right) \right],
\end{equation}
and is dependent on a parameter $k$, ranging from 1 to $N$. The quantity, $\chi$, quantifies the degree of superposition between the state, $\ket{0}^{\bigotimes N}$, and the state, $\ket{1}^{\bigotimes N}$.

In [ ]:
from typing import Tuple, List
from pytket.circuit import Circuit

def generate_parity_circuits(
    ghz_circuit: Circuit,
) -> Tuple[List[Circuit], List[float]]:
    circuits = []
    angles = []
    for k in range(1, ghz_circuit.n_qubits+1):
        circuit = ghz_circuit.copy()
        angle = k / ghz_circuit.n_qubits
        for q in circuit.qubits:
            circuit.Rz(-angle, q)
            circuit.Ry(-0.5, q)
        circuit.measure_all()
        circuits += [circuit]
        angles += [angle]
    return circuits, angles

In [ ]:
parity_circuit_list, angles = generate_parity_circuits(ghz_circuit)

Counting each measurement outcome allows the parity of the bitstring to be computed.

In [ ]:
from typing import List, Dict
from pytket.backends.backendresult import BackendResult

def compute_ghz_parities(
    backend_results: BackendResult,
    angles: List[float]
) -> Dict[float, float]:
    parities = {}
    for (a, b) in zip(angles, backend_results):
        dist = b.get_distribution()
        expectation_value = sum([(-1)**(sum(bitstring)) * prob for bitstring, prob in dist.items()])
        parities[a] = expectation_value
    return parities

The cell below defines a function to combine the population and parity results to estimate the GHZ state fidelity, $F$.

In [ ]:
from typing import List, Dict

def compute_ghz_fidelity(
    parities: List[float], 
    populations: Dict[str, float]
) -> float:
    f_populations = populations.get("zeros", 0) + populations.get("ones", 0)
    f_parities = sum([p * (-1)**k for k, p in enumerate(parities)]) / len(parities)
    return f_populations + f_parities

## Syntax Checking for 56-Qubit Jobs

The circuits are submitted to the syntax checker, `H2-1SC`, to verify the code before execution to the `H2-1` device.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend

qntm_backend_syntax_checker = QuantinuumBackend(device_name="H2-1SC", simulator="stabilizer")
qntm_backend_syntax_checker.login()

Compilation of the circuits to the correct gate set.

In [ ]:
circuits = [population_circuit] + parity_circuit_list
compiled_circuit_list = qntm_backend_syntax_checker.get_compiled_circuits(circuits, optimisation_level=2)

Submission to the syntax-checker.

In [ ]:
sc_handles = qntm_backend_syntax_checker.process_circuits(compiled_circuit_list, n_shots=10)

The status of the circuit can be checked using the result handles from the previous step. If the status is returned as `COMPLETE`, the syntax checker has verified the job is executable on the H-Series device.

In [ ]:
for h in sc_handles:    
    circuit_status = qntm_backend_syntax_checker.circuit_status(h)
    print(circuit_status.status)

## Analysis of GHZ State Fidelity after H2-1 Execution

Execution on `H2-1` leads to a GHZ state fidelity of $0.6156\pm0.0082$.

The results from the population job show the probability of measuring the all-zero and all-one state are between 0.4 and 0.45. The probability of measuring a different state is 0.2. The probability of measuring the all-ones state is worse than all-zeros state due to assymetry in spam error.

<div style="text-align: center;">
    <img src="figures/GHZ_populations.png" width="800" />
</div>

The plot below shows the parity of the GHZ state. This is the degree of superposition in the result. A parity of zero implies no superposition in the GHZ state. Ideally parities should be between -0.5 and 0.5.

<div style="text-align: center;">
    <img src="figures/GHZ_parities.png" width="800" />
</div>

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>